In [1]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import KFold
from itertools import product,chain
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

In [2]:
# read in the train and test data from csv files

train_set = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train_set.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [4]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [5]:
train_set.isnull().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [6]:
test.isnull().sum()

Applicant_ID        0
form_field1      1110
form_field2      1709
form_field3       146
form_field4       146
form_field5       146
form_field6      5604
form_field7      2231
form_field8      5604
form_field9      3400
form_field10      147
form_field11    13398
form_field12     4183
form_field13     2463
form_field14        0
form_field15     9592
form_field16     5474
form_field17     4695
form_field18     4369
form_field19        0
form_field20      147
form_field21     6707
form_field22     8724
form_field23    12125
form_field24     5605
form_field25     2256
form_field26     3172
form_field27     3910
form_field28      147
form_field29      147
form_field30    10908
form_field31    16810
form_field32     2256
form_field33      495
form_field34      147
form_field35     9866
form_field36      903
form_field37     2256
form_field38      147
form_field39     1829
form_field40    18828
form_field41    16349
form_field42      578
form_field43      250
form_field44     2362
form_field

In [7]:
train_set = train_set.fillna(-9999)
test = test.fillna(-9999)

In [8]:
train_set.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,-9999.000000,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,-9999.0,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,-9999.0,956940.0,-9999.0,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,-9999.000000,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,-9999.0,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [9]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,-9999.0000,0.825000,1.01,0.800000,-9999.0,0.0,charge,-9999.000000,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,-9999.000000,0.504974,0.043525
2,Apcnt_1000052,-9999.0,0.27505,0.0600,0.0000,0.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0000,-9999.000000,0.00,-9999.000000,-9999.0,-9999.0,charge,-9999.000000,0.000000,-9999.000000
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,-9999.0000,0.916663,2.02,0.464224,-9999.0,-9999.0,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,-9999.0000,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [10]:
train_set.isnull().sum()

Applicant_ID      0
form_field1       0
form_field2       0
form_field3       0
form_field4       0
form_field5       0
form_field6       0
form_field7       0
form_field8       0
form_field9       0
form_field10      0
form_field11      0
form_field12      0
form_field13      0
form_field14      0
form_field15      0
form_field16      0
form_field17      0
form_field18      0
form_field19      0
form_field20      0
form_field21      0
form_field22      0
form_field23      0
form_field24      0
form_field25      0
form_field26      0
form_field27      0
form_field28      0
form_field29      0
form_field30      0
form_field31      0
form_field32      0
form_field33      0
form_field34      0
form_field35      0
form_field36      0
form_field37      0
form_field38      0
form_field39      0
form_field40      0
form_field41      0
form_field42      0
form_field43      0
form_field44      0
form_field45      0
form_field46      0
form_field47      0
form_field48      0
form_field49      0


In [11]:
test.isnull().sum()

Applicant_ID    0
form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [12]:
# convert categorical columns to integers
category_cols = ['form_field47', 'default_status']
for column in category_cols:
    train_set[column] = train_set[column].astype('category').cat.codes
    test['form_field47'] = test['form_field47'].astype('category').cat.codes

In [13]:
train_set.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,0,-9999.000000,1.129518,0.044335,0
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,-9999.0,0.0,0,349.805730,1.620483,0.322436,0
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,-9999.0,956940.0,-9999.0,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,0,-9999.000000,1.513370,0.011640,1
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,-9999.0,0.0,0,89.940100,0.664452,0.082729,0
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,0,97.887502,1.427891,0.045630,0


In [14]:
# split labels out of data sets    
train_label = train_set['default_status']
train_set = train_set.drop(['Applicant_ID','default_status'], axis=1) # remove labels
test_set = test.drop('Applicant_ID', axis=1) # dropping the first column because it has unique elements for all entries

In [15]:
x_train, x_test, y_train, y_test = train_test_split(train_set, train_label, test_size = 0.3)

In [16]:
clf = cb.CatBoostClassifier(n_estimators = 3000, early_stopping_rounds = 50, learning_rate = 0.01, random_seed =50)

In [17]:
# train default classifier    
clf.fit(x_train, y_train)

0:	learn: 0.6862946	total: 175ms	remaining: 8m 45s
1:	learn: 0.6797321	total: 284ms	remaining: 7m 4s
2:	learn: 0.6734897	total: 420ms	remaining: 6m 59s
3:	learn: 0.6671787	total: 726ms	remaining: 9m 3s
4:	learn: 0.6611484	total: 883ms	remaining: 8m 48s
5:	learn: 0.6557531	total: 1.11s	remaining: 9m 16s
6:	learn: 0.6500182	total: 1.26s	remaining: 9m
7:	learn: 0.6447497	total: 1.49s	remaining: 9m 15s
8:	learn: 0.6393850	total: 1.69s	remaining: 9m 20s
9:	learn: 0.6341897	total: 1.8s	remaining: 8m 58s
10:	learn: 0.6288848	total: 1.97s	remaining: 8m 54s
11:	learn: 0.6238958	total: 2.11s	remaining: 8m 46s
12:	learn: 0.6191267	total: 2.36s	remaining: 9m 2s
13:	learn: 0.6144550	total: 2.61s	remaining: 9m 16s
14:	learn: 0.6098867	total: 2.7s	remaining: 8m 58s
15:	learn: 0.6057981	total: 2.98s	remaining: 9m 16s
16:	learn: 0.6015986	total: 3.19s	remaining: 9m 19s
17:	learn: 0.5974011	total: 3.38s	remaining: 9m 20s
18:	learn: 0.5934746	total: 3.59s	remaining: 9m 22s
19:	learn: 0.5892150	total: 3.7

159:	learn: 0.4277439	total: 30s	remaining: 8m 53s
160:	learn: 0.4274692	total: 30.3s	remaining: 8m 53s
161:	learn: 0.4272148	total: 30.4s	remaining: 8m 51s
162:	learn: 0.4269391	total: 30.6s	remaining: 8m 52s
163:	learn: 0.4266936	total: 30.8s	remaining: 8m 52s
164:	learn: 0.4264632	total: 30.9s	remaining: 8m 51s
165:	learn: 0.4262491	total: 31.2s	remaining: 8m 51s
166:	learn: 0.4260256	total: 31.3s	remaining: 8m 50s
167:	learn: 0.4257895	total: 31.5s	remaining: 8m 50s
168:	learn: 0.4255788	total: 31.6s	remaining: 8m 49s
169:	learn: 0.4253513	total: 31.8s	remaining: 8m 49s
170:	learn: 0.4251230	total: 32s	remaining: 8m 49s
171:	learn: 0.4248936	total: 32.1s	remaining: 8m 47s
172:	learn: 0.4246877	total: 32.3s	remaining: 8m 47s
173:	learn: 0.4245068	total: 32.5s	remaining: 8m 48s
174:	learn: 0.4242963	total: 32.6s	remaining: 8m 46s
175:	learn: 0.4240473	total: 32.8s	remaining: 8m 45s
176:	learn: 0.4238574	total: 33s	remaining: 8m 45s
177:	learn: 0.4236943	total: 33.1s	remaining: 8m 44s

315:	learn: 0.4088342	total: 57.4s	remaining: 8m 7s
316:	learn: 0.4087749	total: 57.5s	remaining: 8m 6s
317:	learn: 0.4087254	total: 57.7s	remaining: 8m 6s
318:	learn: 0.4086664	total: 57.8s	remaining: 8m 5s
319:	learn: 0.4086093	total: 58s	remaining: 8m 5s
320:	learn: 0.4085623	total: 58.1s	remaining: 8m 4s
321:	learn: 0.4085199	total: 58.2s	remaining: 8m 4s
322:	learn: 0.4084554	total: 58.4s	remaining: 8m 3s
323:	learn: 0.4084041	total: 58.6s	remaining: 8m 3s
324:	learn: 0.4083551	total: 58.8s	remaining: 8m 3s
325:	learn: 0.4082923	total: 58.9s	remaining: 8m 2s
326:	learn: 0.4082382	total: 59s	remaining: 8m 2s
327:	learn: 0.4081869	total: 59.2s	remaining: 8m 2s
328:	learn: 0.4081394	total: 59.4s	remaining: 8m 2s
329:	learn: 0.4080815	total: 59.6s	remaining: 8m 1s
330:	learn: 0.4080188	total: 59.8s	remaining: 8m 2s
331:	learn: 0.4079609	total: 1m	remaining: 8m 2s
332:	learn: 0.4079123	total: 1m	remaining: 8m 2s
333:	learn: 0.4078553	total: 1m	remaining: 8m 1s
334:	learn: 0.4078075	tot

471:	learn: 0.4017230	total: 1m 25s	remaining: 7m 40s
472:	learn: 0.4016871	total: 1m 26s	remaining: 7m 39s
473:	learn: 0.4016574	total: 1m 26s	remaining: 7m 39s
474:	learn: 0.4016314	total: 1m 26s	remaining: 7m 39s
475:	learn: 0.4015994	total: 1m 26s	remaining: 7m 39s
476:	learn: 0.4015580	total: 1m 26s	remaining: 7m 39s
477:	learn: 0.4015296	total: 1m 27s	remaining: 7m 39s
478:	learn: 0.4014884	total: 1m 27s	remaining: 7m 39s
479:	learn: 0.4014505	total: 1m 27s	remaining: 7m 39s
480:	learn: 0.4014104	total: 1m 27s	remaining: 7m 39s
481:	learn: 0.4013816	total: 1m 27s	remaining: 7m 39s
482:	learn: 0.4013512	total: 1m 27s	remaining: 7m 38s
483:	learn: 0.4013296	total: 1m 28s	remaining: 7m 38s
484:	learn: 0.4012897	total: 1m 28s	remaining: 7m 37s
485:	learn: 0.4012697	total: 1m 28s	remaining: 7m 36s
486:	learn: 0.4012363	total: 1m 28s	remaining: 7m 36s
487:	learn: 0.4012071	total: 1m 28s	remaining: 7m 36s
488:	learn: 0.4011786	total: 1m 28s	remaining: 7m 36s
489:	learn: 0.4011474	total:

625:	learn: 0.3971239	total: 1m 52s	remaining: 7m 5s
626:	learn: 0.3971011	total: 1m 52s	remaining: 7m 4s
627:	learn: 0.3970677	total: 1m 52s	remaining: 7m 4s
628:	learn: 0.3970475	total: 1m 52s	remaining: 7m 4s
629:	learn: 0.3970316	total: 1m 52s	remaining: 7m 4s
630:	learn: 0.3970072	total: 1m 52s	remaining: 7m 3s
631:	learn: 0.3969718	total: 1m 53s	remaining: 7m 3s
632:	learn: 0.3969451	total: 1m 53s	remaining: 7m 3s
633:	learn: 0.3969191	total: 1m 53s	remaining: 7m 3s
634:	learn: 0.3968948	total: 1m 53s	remaining: 7m 3s
635:	learn: 0.3968612	total: 1m 53s	remaining: 7m 2s
636:	learn: 0.3968351	total: 1m 53s	remaining: 7m 2s
637:	learn: 0.3968087	total: 1m 54s	remaining: 7m 2s
638:	learn: 0.3967953	total: 1m 54s	remaining: 7m 2s
639:	learn: 0.3967687	total: 1m 54s	remaining: 7m 1s
640:	learn: 0.3967481	total: 1m 54s	remaining: 7m 1s
641:	learn: 0.3967289	total: 1m 54s	remaining: 7m 1s
642:	learn: 0.3967105	total: 1m 54s	remaining: 7m
643:	learn: 0.3966871	total: 1m 54s	remaining: 7m

...

2291:	learn: 0.3604241	total: 6m 42s	remaining: 2m 4s
2292:	learn: 0.3604036	total: 6m 42s	remaining: 2m 3s
2293:	learn: 0.3603975	total: 6m 42s	remaining: 2m 3s
2294:	learn: 0.3603850	total: 6m 42s	remaining: 2m 3s
2295:	learn: 0.3603622	total: 6m 42s	remaining: 2m 3s
2296:	learn: 0.3603395	total: 6m 42s	remaining: 2m 3s
2297:	learn: 0.3603183	total: 6m 42s	remaining: 2m 3s
2298:	learn: 0.3603045	total: 6m 42s	remaining: 2m 2s
2299:	learn: 0.3602888	total: 6m 43s	remaining: 2m 2s
2300:	learn: 0.3602802	total: 6m 43s	remaining: 2m 2s
2301:	learn: 0.3602567	total: 6m 43s	remaining: 2m 2s
2302:	learn: 0.3602412	total: 6m 43s	remaining: 2m 2s
2303:	learn: 0.3602233	total: 6m 43s	remaining: 2m 1s
2304:	learn: 0.3602022	total: 6m 44s	remaining: 2m 1s
2305:	learn: 0.3601793	total: 6m 44s	remaining: 2m 1s
2306:	learn: 0.3601510	total: 6m 44s	remaining: 2m 1s
2307:	learn: 0.3601249	total: 6m 44s	remaining: 2m 1s
2308:	learn: 0.3601057	total: 6m 44s	remaining: 2m 1s
2309:	learn: 0.3600791	total

2443:	learn: 0.3575235	total: 7m 6s	remaining: 1m 37s
2444:	learn: 0.3575061	total: 7m 7s	remaining: 1m 36s
2445:	learn: 0.3574871	total: 7m 7s	remaining: 1m 36s
2446:	learn: 0.3574638	total: 7m 7s	remaining: 1m 36s
2447:	learn: 0.3574476	total: 7m 7s	remaining: 1m 36s
2448:	learn: 0.3574234	total: 7m 7s	remaining: 1m 36s
2449:	learn: 0.3574075	total: 7m 8s	remaining: 1m 36s
2450:	learn: 0.3573915	total: 7m 8s	remaining: 1m 35s
2451:	learn: 0.3573757	total: 7m 8s	remaining: 1m 35s
2452:	learn: 0.3573562	total: 7m 8s	remaining: 1m 35s
2453:	learn: 0.3573398	total: 7m 8s	remaining: 1m 35s
2454:	learn: 0.3573181	total: 7m 9s	remaining: 1m 35s
2455:	learn: 0.3572921	total: 7m 9s	remaining: 1m 35s
2456:	learn: 0.3572744	total: 7m 9s	remaining: 1m 34s
2457:	learn: 0.3572575	total: 7m 9s	remaining: 1m 34s
2458:	learn: 0.3572365	total: 7m 9s	remaining: 1m 34s
2459:	learn: 0.3572133	total: 7m 9s	remaining: 1m 34s
2460:	learn: 0.3571941	total: 7m 10s	remaining: 1m 34s
2461:	learn: 0.3571719	tota

2593:	learn: 0.3546310	total: 7m 32s	remaining: 1m 10s
2594:	learn: 0.3546109	total: 7m 32s	remaining: 1m 10s
2595:	learn: 0.3545895	total: 7m 32s	remaining: 1m 10s
2596:	learn: 0.3545780	total: 7m 33s	remaining: 1m 10s
2597:	learn: 0.3545528	total: 7m 33s	remaining: 1m 10s
2598:	learn: 0.3545318	total: 7m 33s	remaining: 1m 9s
2599:	learn: 0.3545078	total: 7m 33s	remaining: 1m 9s
2600:	learn: 0.3544888	total: 7m 33s	remaining: 1m 9s
2601:	learn: 0.3544766	total: 7m 33s	remaining: 1m 9s
2602:	learn: 0.3544522	total: 7m 34s	remaining: 1m 9s
2603:	learn: 0.3544285	total: 7m 34s	remaining: 1m 9s
2604:	learn: 0.3544099	total: 7m 34s	remaining: 1m 8s
2605:	learn: 0.3543951	total: 7m 34s	remaining: 1m 8s
2606:	learn: 0.3543744	total: 7m 34s	remaining: 1m 8s
2607:	learn: 0.3543602	total: 7m 34s	remaining: 1m 8s
2608:	learn: 0.3543365	total: 7m 35s	remaining: 1m 8s
2609:	learn: 0.3543168	total: 7m 35s	remaining: 1m 8s
2610:	learn: 0.3542927	total: 7m 35s	remaining: 1m 7s
2611:	learn: 0.3542687	

2746:	learn: 0.3517697	total: 7m 57s	remaining: 44s
2747:	learn: 0.3517467	total: 7m 58s	remaining: 43.8s
2748:	learn: 0.3517256	total: 7m 58s	remaining: 43.7s
2749:	learn: 0.3517062	total: 7m 58s	remaining: 43.5s
2750:	learn: 0.3516869	total: 7m 58s	remaining: 43.3s
2751:	learn: 0.3516668	total: 7m 58s	remaining: 43.1s
2752:	learn: 0.3516465	total: 7m 58s	remaining: 43s
2753:	learn: 0.3516397	total: 7m 59s	remaining: 42.8s
2754:	learn: 0.3516153	total: 7m 59s	remaining: 42.6s
2755:	learn: 0.3515908	total: 7m 59s	remaining: 42.4s
2756:	learn: 0.3515777	total: 7m 59s	remaining: 42.3s
2757:	learn: 0.3515540	total: 7m 59s	remaining: 42.1s
2758:	learn: 0.3515320	total: 7m 59s	remaining: 41.9s
2759:	learn: 0.3515193	total: 8m	remaining: 41.7s
2760:	learn: 0.3514955	total: 8m	remaining: 41.6s
2761:	learn: 0.3514853	total: 8m	remaining: 41.4s
2762:	learn: 0.3514755	total: 8m	remaining: 41.2s
2763:	learn: 0.3514608	total: 8m	remaining: 41s
2764:	learn: 0.3514417	total: 8m	remaining: 40.9s
2765

2900:	learn: 0.3488818	total: 8m 23s	remaining: 17.2s
2901:	learn: 0.3488602	total: 8m 24s	remaining: 17s
2902:	learn: 0.3488463	total: 8m 24s	remaining: 16.8s
2903:	learn: 0.3488249	total: 8m 24s	remaining: 16.7s
2904:	learn: 0.3488022	total: 8m 24s	remaining: 16.5s
2905:	learn: 0.3487858	total: 8m 24s	remaining: 16.3s
2906:	learn: 0.3487702	total: 8m 24s	remaining: 16.2s
2907:	learn: 0.3487552	total: 8m 25s	remaining: 16s
2908:	learn: 0.3487373	total: 8m 25s	remaining: 15.8s
2909:	learn: 0.3487224	total: 8m 25s	remaining: 15.6s
2910:	learn: 0.3487029	total: 8m 25s	remaining: 15.5s
2911:	learn: 0.3486812	total: 8m 25s	remaining: 15.3s
2912:	learn: 0.3486670	total: 8m 25s	remaining: 15.1s
2913:	learn: 0.3486507	total: 8m 25s	remaining: 14.9s
2914:	learn: 0.3486326	total: 8m 25s	remaining: 14.8s
2915:	learn: 0.3486104	total: 8m 26s	remaining: 14.6s
2916:	learn: 0.3485848	total: 8m 26s	remaining: 14.4s
2917:	learn: 0.3485605	total: 8m 26s	remaining: 14.2s
2918:	learn: 0.3485395	total: 8m

In [18]:
roc_auc_score(y_test, clf.predict_proba(x_test)[:,1])

0.8346197846311179

In [19]:
pred = clf.predict_proba(test_set)[:,1]

In [20]:
prediction = pd.DataFrame({'Applicant_ID': test['Applicant_ID'], 'default_status': pred})
prediction.head()

,Applicant_ID,default_status
0,Apcnt_1000032,0.305414
1,Apcnt_1000048,0.479333
2,Apcnt_1000052,0.408900
3,Apcnt_1000076,0.810406
4,Apcnt_1000080,0.199643


In [21]:
prediction.to_csv('Submit99.csv', index = False)